In [1]:
print("hello world")
#hello can you see this new change
#please write your name just to make sure we can all edit this
#bea
#dilara
#gabi

hello world


In [2]:
#Imports
import numpy as np
import scipy as sci

In [3]:
"""
Class for quantum register for n number of qubits
"""
def tensor_Product(a,b):
    product= a+b
    return product

class Quantum_Register:
    def __init__(self,n):
        self.n = n 
        self.psi = np.zeros((2,)*n)

In [4]:
"""
Class for the quantum circuits 
"""
class Quantum_Circuit:
    def __init__(self):
        self = self

In [5]:
"""Testing block """
print("the tensor product is, ", tensor_Product(1,2))
newRegister= Quantum_Register(1)
print(newRegister.n)
print(newRegister.psi)

the tensor product is,  3
1
[0. 0.]


In [16]:
#define number of qubits, N

N = 12

In [17]:
def qubit_register(N):
    psi = np.zeros((2**N, 1), dtype = complex)
    psi[0] = 1/(2**(1/2))
    psi[2**N-1] = 1/(2**(1/2))
    
    return psi
psi = qubit_register(N)
print(psi)

[[0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]
 ...
 [0.        +0.j]
 [0.        +0.j]
 [0.70710678+0.j]]
